In [ ]:
import os 
import json
import agent_tools as t
from agent_tools import save_structure
from openai import AsyncOpenAI 
import handoff_instructions
import instructions
from agents import Agent, Runner, OpenAIChatCompletionsModel, trace

In [ ]:
# import environment variables
from dotenv import load_dotenv
load_dotenv(override=True)

In [ ]:
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')

In [ ]:
if not deepseek_api_key:
    print(f"Missing deepseek api key")
else:
    print(f"Deepseek api key: {deepseek_api_key[:8]} ... {deepseek_api_key[-8:]}")

In [ ]:
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"

In [ ]:
# Our models 
openai_model_mini = "gpt-4o-mini"
openai_model_gpt5 = "gpt-5-nano"
deepseek_model_reasoner = "deepseek-reasoner"

In [ ]:
# Skipping...
deepseek_client = AsyncOpenAI(api_key=deepseek_api_key,base_url=DEEPSEEK_BASE_URL)

In [ ]:
# Skipping deepseek model, not compatible with openai/cant' runt tools 
deepseek_model = OpenAIChatCompletionsModel(model=deepseek_model_reasoner,openai_client=deepseek_client)

In [ ]:
""" Innovator agent comes up with innovative ideas,to help 
solve major problems, for small business."""

innovator_agent = Agent(
    name="Idea Generator Agent",
    instructions=instructions.innovator_instructions, 
    model=openai_model_mini
    )

In [ ]:
""" Idea Selector Agent selects the most effective idea from a list of ideas, 
selects the one that addresses user's pain points, considering cost"""

idea_selector_agent = Agent(
    name="Idea Selector Agent",
    instructions=instructions.idea_selector_instructions,
    model=openai_model_mini
    )

In [ ]:
""" Create app structure for the selected idea"""
app_structure_agent = Agent(
    name='App Structure Agent',
    instructions=instructions.app_file_structure_instructions,
    model=openai_model_gpt5
)

In [ ]:

""" Developer Agent writes the backend source code for the app """
developer_agent = Agent(
    name='Developer Agent', 
    instructions=instructions.developer_instructions,
    model=openai_model_gpt5,
    tools=[t.save_source_code]
)

In [ ]:
""" Database Creator Agent designs the database schema and relationships."""
database_creator_agent = Agent(
    name='Database Designer Agent',
    instructions=instructions.database_instructions,
    model=openai_model_gpt5,
    tools=[t.save_database_content]
)

In [ ]:
""" Let's skip QA for now, since the model takes long time to run"""
qa_tester_agent = Agent(
    name="Tester Agent",
    instructions=instructions.qa_tester_instructions,
    model=openai_model_mini
)

In [ ]:
""" Double checks app structure for correctness"""
structure_verifier_agent = Agent(
    name="JSON Structure Verifier",
    instructions=instructions.structure_verifier_instructions,
    model=openai_model_gpt5, 
)

In [ ]:
# Convert all agents as tools for manager agent
ideas_tool = innovator_agent.as_tool(
    tool_name="Idea_Tool",
    tool_description="Use this tool to help you come up with list of ideas, for small businesses."
    )
idea_selector_tool = idea_selector_agent.as_tool(
    tool_name="Idea_Selector_Tool",
    tool_description=f"Use this tool to select the best idea from  generated list of ideas."
    )
app_structure_tool = app_structure_agent.as_tool(
    tool_name="App_Structure_Tool",
    tool_description=f"Use this tool to help you create a correct and complete React app structure for the selected idea."
    )
database_tool = database_creator_agent.as_tool(
    tool_name="Database_Designer_Tool",
    tool_description="Use this tool to help you write code for creating database files/schemas."
    )
"""
structure_verifier_agent.as_tool(
    tool_name="JSON Structure Verifier",
    tool_description="Use this tool to double check the JSON structure is correct format and error free"
)"""

dev_tool = developer_agent.as_tool(
    tool_name="Developer_Tool",
    tool_description="Use this tool after you have gathered all the required information, to help you write all the React app source code for the selected idea."
)


In [ ]:
manager_tools = [
    ideas_tool,
    idea_selector_tool,
    app_structure_tool,
    database_tool,
    dev_tool,
    ]

In [ ]:
"""Equip manger agent with tools """
manager_agent = Agent(
    name='Manager Agent',
    instructions=instructions.manager_instructions, 
    model=openai_model_gpt5,
    tools=manager_tools,
)

In [ ]:
# Pass the ideas to the agent so it can select the best one
innovator_agent.handoffs = [manager_agent, developer_agent]
innovator_agent.handoff_description = handoff_instructions.innovator_handoff

# Create the app file structure for the selected idea
idea_selector_agent.handoffs = [manager_agent, developer_agent]
idea_selector_agent.handoff_description = handoff_instructions.idea_selector_handoff

# Create structure for the selected app 
app_structure_agent.handoffs = [manager_agent, developer_agent]
app_structure_agent.handoff_description = handoff_instructions.app_structure_handoff

"""
# Double check the structure, make sure correct key and values with correct content
structure_verifier_agent.handoffs = [developer_agent]
structure_verifier_agent.handoff_description = handoff_instructions.structure_verifier_handoff
"""

# Database Creator Agent handoffs to Developer Agent its work
database_creator_agent.handoffs = [manager_agent, developer_agent]
database_creator_agent.handoff_description = handoff_instructions.database_handoff

qa_tester_agent.handoffs = [manager_agent]
qa_tester_agent.handoff_description = handoff_instructions.qa_tester_handoff 

developer_agent.handoffs = [manager_agent]
developer_agent.handoff_description = handoff_instructions.developer_handoff


In [ ]:
"""It's show time"""
with trace('Manager Agent Run'):
    result = await Runner.run(
        starting_agent=manager_agent, 
        input="Come up with an innovative idea, then build the idea using your tools.",
        max_turns=10
    )

print(result.final_output)

In [ ]:
""" Create app """
with open("app_code.json", "r") as f:
        project_structure = json.load(f)
        print(project_structure)
        save_structure(project_structure)